# CNN With PyTorch

## Source: [Blog](https://adventuresinmachinelearning.com/convolutional-neural-networks-tutorial-in-pytorch/)

## Hyperparameters

In [4]:
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001
out_dir = "./dataset"

## Importing Packages

In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

## Loading and Preprocessing MNIST Dataset

In [9]:
compose = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,),(0.3081,))])

train_dataset = torchvision.datasets.MNIST(root=out_dir, train=True, transform=compose, download=True)
test_dataset = torchvision.datasets.MNIST(root=out_dir, train=False, transform=compose, download=True)


Extracting ./dataset/MNIST/raw/train-images-idx3-ubyte.gz to ./dataset/MNIST/raw



Extracting ./dataset/MNIST/raw/train-labels-idx1-ubyte.gz to ./dataset/MNIST/raw



Extracting ./dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to ./dataset/MNIST/raw



Extracting ./dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./dataset/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:479: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## Creating DataLoader for Training and Testing

In [10]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

## Creating the ConvNet class for Model Instantiation

In [26]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    self.layer1 = nn.Sequential(
      nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2, stride=2)  
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.drop_out = nn.Dropout()
    self.fc1 = nn.Linear(7 * 7 * 64, 1000)
    self.fc2 = nn.Linear(1000, 10)

  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.reshape(out.size(0), -1) # Understand!
    out = self.drop_out(out)
    out = self.fc1(out)
    out = self.fc2(out)
    return out

## Initialising model and loss

In [27]:
model = ConvNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Training the model

In [28]:
total_step = len(train_loader)

loss_list = []
acc_list = []
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    outputs = model(images)
    loss = criterion(outputs, labels)
    loss_list.append(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total = labels.size(0)
    _, predicted = torch.max(outputs.data, 1)
    correct = (predicted == labels).sum().item()
    acc_list.append(correct/total)

    if( i + 1) % 100 == 0:
      print("Epoch [{} / {}], Step [{}/ {}], Loss: {:.4f}, Accuracy: {:.2f}%".format(epoch + 1, num_epochs, i + 1, total_step, loss.item(), (correct/total) * 100 ))

Epoch [1 / 5], Step [100/ 600], Loss: 0.2300, Accuracy: 91.00%
Epoch [1 / 5], Step [200/ 600], Loss: 0.1070, Accuracy: 94.00%
Epoch [1 / 5], Step [300/ 600], Loss: 0.0737, Accuracy: 97.00%
Epoch [1 / 5], Step [400/ 600], Loss: 0.1060, Accuracy: 96.00%
Epoch [1 / 5], Step [500/ 600], Loss: 0.0876, Accuracy: 99.00%
Epoch [1 / 5], Step [600/ 600], Loss: 0.0783, Accuracy: 97.00%
Epoch [2 / 5], Step [100/ 600], Loss: 0.0432, Accuracy: 98.00%
Epoch [2 / 5], Step [200/ 600], Loss: 0.0555, Accuracy: 97.00%
Epoch [2 / 5], Step [300/ 600], Loss: 0.0466, Accuracy: 98.00%
Epoch [2 / 5], Step [400/ 600], Loss: 0.0625, Accuracy: 99.00%
Epoch [2 / 5], Step [500/ 600], Loss: 0.0439, Accuracy: 98.00%
Epoch [2 / 5], Step [600/ 600], Loss: 0.0700, Accuracy: 97.00%
Epoch [3 / 5], Step [100/ 600], Loss: 0.0287, Accuracy: 99.00%
Epoch [3 / 5], Step [200/ 600], Loss: 0.0233, Accuracy: 99.00%
Epoch [3 / 5], Step [300/ 600], Loss: 0.0286, Accuracy: 100.00%
Epoch [3 / 5], Step [400/ 600], Loss: 0.0076, Accuracy

## Testing the model

In [37]:
model.eval()
with torch.no_grad():
  correct = 0
  total = 0
  for images, labels in test_loader:
    outputs = model(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

  print("Test Accuracy of the model on the 10000 test images: {} %".format((correct/total)*100))

Test Accuracy of the model on the 10000 test images: 99.13 %
